In [ ]:
import tkinter as tk
import numpy as np
import random
import time
from queue import Queue, PriorityQueue

# Initialize global variables
CANVAS_SIZE = 600
SMALL_MAZE = 10
MEDIUM_MAZE = 20
LARGE_MAZE = 30

class MazeSolverApp:
    def __init__(self, root):
        self.root = root
        self.root.title("AI Maze Solver with Heuristics")

        # Create canvas for visualization
        self.canvas = tk.Canvas(self.root, width=CANVAS_SIZE, height=CANVAS_SIZE, bg='white')
        self.canvas.grid(row=0, column=0, columnspan=8)

        # Add buttons for algorithms and heuristics
        self.random_button = tk.Button(self.root, text="Random Maze", command=self.generate_random_maze)
        self.random_button.grid(row=1, column=0)

        self.bfs_button = tk.Button(self.root, text="Solve with BFS", command=lambda: self.solve_with_algorithm(bfs, "BFS"))
        self.bfs_button.grid(row=1, column=1)

        self.dfs_button = tk.Button(self.root, text="Solve with DFS", command=lambda: self.solve_with_algorithm(dfs, "DFS"))
        self.dfs_button.grid(row=1, column=2)

        self.astar_manhattan_button = tk.Button(self.root, text="A* (Manhattan)", command=lambda: self.solve_with_algorithm(lambda maze: a_star(maze, manhattan_heuristic), "A* (Manhattan)"))
        self.astar_manhattan_button.grid(row=1, column=3)

        self.astar_euclidean_button = tk.Button(self.root, text="A* (Euclidean)", command=lambda: self.solve_with_algorithm(lambda maze: a_star(maze, euclidean_heuristic), "A* (Euclidean)"))
        self.astar_euclidean_button.grid(row=1, column=4)

        self.bidirectional_button = tk.Button(self.root, text="Bidirectional Search", command=lambda: self.solve_with_algorithm(bidirectional_search, "Bidirectional Search"))
        self.bidirectional_button.grid(row=1, column=5)

        self.ucs_button = tk.Button(self.root, text="UCS", command=lambda: self.solve_with_algorithm(lambda maze: a_star(maze, uniform_cost_heuristic), "UCS"))
        self.ucs_button.grid(row=1, column=6)

        self.size_label = tk.Label(self.root, text="Maze Size:")
        self.size_label.grid(row=1, column=7)

        self.size_var = tk.StringVar(value="Small")
        self.size_menu = tk.OptionMenu(self.root, self.size_var, "Small", "Medium", "Large", command=self.change_size)
        self.size_menu.grid(row=1, column=8)

        # Initialize maze and visualization
        self.maze_size = SMALL_MAZE
        self.maze = self.generate_maze(self.maze_size)
        self.solution_path = None
        self.explored_path = []
        self.draw_maze()

    def change_size(self, size_text):
        """Change maze size based on user selection."""
        if size_text == "Small":
            self.maze_size = SMALL_MAZE
        elif size_text == "Medium":
            self.maze_size = MEDIUM_MAZE
        else:
            self.maze_size = LARGE_MAZE

        self.generate_random_maze()

    def generate_random_maze(self):
        """Generate a new random maze."""
        self.maze = self.generate_maze(self.maze_size)
        self.solution_path = None
        self.explored_path = []
        self.draw_maze()

    def generate_maze(self, size):
        """Generate a solvable maze with random walls."""
        maze = np.ones((size, size), dtype=int)
        maze[0][0] = 0
        maze[size - 1][size - 1] = 0

        def dfs_generate(x, y):
            directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
            random.shuffle(directions)
            for dx, dy in directions:
                nx, ny = x + dx, y + dy
                if 0 <= nx < size and 0 <= ny < size and maze[nx][ny] == 1:
                    maze[nx][ny] = 0
                    dfs_generate(nx, ny)

        dfs_generate(0, 0)
        return maze

    def draw_maze(self):
        """Draw the maze grid."""
        self.canvas.delete("all")
        size = len(self.maze)
        cell_size = CANVAS_SIZE // size
        for i in range(size):
            for j in range(size):
                color = 'white' if self.maze[i][j] == 0 else 'black'
                if (i, j) == (0, 0):
                    color = 'green'
                elif (i, j) == (size - 1, size - 1):
                    color = 'red'
                self.canvas.create_rectangle(j * cell_size, i * cell_size, (j + 1) * cell_size, (i + 1) * cell_size, fill=color)

    def solve_with_algorithm(self, algorithm, name):
        """Solve the maze using the specified algorithm."""
        start_time = time.perf_counter()
        self.explored_path, self.solution_path = algorithm(self.maze)
        elapsed_time = time.perf_counter() - start_time

        if self.solution_path:
            print(f"{name} solved the maze in {elapsed_time:.6f} seconds.")
            self.draw_solution()
        else:
            print(f"{name} could not find a solution.")

    def draw_solution(self):
        """Visualize the solution path."""
        size = len(self.maze)
        cell_size = CANVAS_SIZE // size
        for i, j in self.solution_path:
            self.canvas.create_rectangle(j * cell_size, i * cell_size, (j + 1) * cell_size, (i + 1) * cell_size, fill='blue')
            self.root.update()
            time.sleep(0.05)

# Heuristic functions
def manhattan_heuristic(pos, end):
    return abs(pos[0] - end[0]) + abs(pos[1] - end[1])

def euclidean_heuristic(pos, end):
    return ((pos[0] - end[0]) ** 2 + (pos[1] - end[1]) ** 2) ** 0.5

def uniform_cost_heuristic(pos, end):
    return 0  # UCS does not use heuristics

# Search algorithms
def dfs(maze):
    start = (0, 0)
    end = (len(maze) - 1, len(maze) - 1)
    stack = [start]
    parent = {}
    visited = set()
    explored_path = []

    while stack:
        current = stack.pop()
        explored_path.append(current)
        if current == end:
            return explored_path, reconstruct_path(parent, start, end)

        if current not in visited:
            visited.add(current)
            for neighbor in get_neighbors(current, maze):
                if neighbor not in visited:
                    stack.append(neighbor)
                    parent[neighbor] = current
    return explored_path, None

def bfs(maze):
    start = (0, 0)
    end = (len(maze) - 1, len(maze) - 1)
    queue = Queue()
    queue.put(start)
    parent = {}
    visited = {start}
    explored_path = []

    while not queue.empty():
        current = queue.get()
        explored_path.append(current)
        if current == end:
            return explored_path, reconstruct_path(parent, start, end)

        for neighbor in get_neighbors(current, maze):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.put(neighbor)
                parent[neighbor] = current
    return explored_path, None

def a_star(maze, heuristic):
    start = (0, 0)
    end = (len(maze) - 1, len(maze) - 1)
    pq = PriorityQueue()
    pq.put((0, start))
    parent = {}
    visited = set()
    g_cost = {start: 0}
    explored_path = []

    while not pq.empty():
        _, current = pq.get()
        explored_path.append(current)
        if current == end:
            return explored_path, reconstruct_path(parent, start, end)

        if current not in visited:
            visited.add(current)
            for neighbor in get_neighbors(current, maze):
                tentative_g_cost = g_cost[current] + 1
                if neighbor not in g_cost or tentative_g_cost < g_cost[neighbor]:
                    g_cost[neighbor] = tentative_g_cost
                    f_cost = tentative_g_cost + heuristic(neighbor, end)
                    pq.put((f_cost, neighbor))
                    parent[neighbor] = current
    return explored_path, None

def bidirectional_search(maze):
    start = (0, 0)
    end = (len(maze) - 1, len(maze) - 1)
    queue_start = Queue()
    queue_end = Queue()
    queue_start.put(start)
    queue_end.put(end)
    visited_start = {start}
    visited_end = {end}
    parent_start = {start: None}
    parent_end = {end: None}
    explored_path_start = []
    explored_path_end = []

    def reconstruct_full_path(intersect):
        path_start = []
        current = intersect
        while current is not None:
            path_start.append(current)
            current = parent_start[current]
        path_start.reverse()

        path_end = []
        current = intersect
        while current is not None:
            path_end.append(current)
            current = parent_end[current]

        path_end = path_end[1:]
        return path_start + path_end

    while not queue_start.empty() and not queue_end.empty():
        current_start = queue_start.get()
        explored_path_start.append(current_start)

        for neighbor in get_neighbors(current_start, maze):
            if neighbor not in visited_start:
                visited_start.add(neighbor)
                parent_start[neighbor] = current_start
                queue_start.put(neighbor)

                if neighbor in visited_end:
                    return explored_path_start + explored_path_end, reconstruct_full_path(neighbor)

        current_end = queue_end.get()
        explored_path_end.append(current_end)

        for neighbor in get_neighbors(current_end, maze):
            if neighbor not in visited_end:
                visited_end.add(neighbor)
                parent_end[neighbor] = current_end
                queue_end.put(neighbor)

                if neighbor in visited_start:
                    return explored_path_start + explored_path_end, reconstruct_full_path(neighbor)

    return explored_path_start + explored_path_end, None

def reconstruct_path(parent, start, end):
    path = []
    while end:
        path.append(end)
        end = parent.get(end)
    return path[::-1]

def get_neighbors(pos, maze):
    row, col = pos
    neighbors = []
    if row > 0 and maze[row - 1][col] == 0:
        neighbors.append((row - 1, col))
    if row < len(maze) - 1 and maze[row + 1][col] == 0:
        neighbors.append((row + 1, col))
    if col > 0 and maze[row][col - 1] == 0:
        neighbors.append((row, col - 1))
    if col < len(maze[0]) - 1 and maze[row][col + 1] == 0:
        neighbors.append((row, col + 1))
    return neighbors

if __name__ == "__main__":
    root = tk.Tk()
    app = MazeSolverApp(root)
    root.mainloop()
